In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [3]:
# load the dataset
max_features = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}")

17464789/17464789 [==============================] - 6s 0us/step
Training data shape: (25000,), Training labels shape: (25000,)
Testing data shape: (25000,), Testing labels shape: (25000,)


In [5]:
sample_review = X_train[0]
sample_review # the reviews are already converted to one hot vector representation

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [6]:
sample_label = y_train[0]
sample_label

1

In [7]:
# max len of sentence
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [10]:
# Train simple RNN model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len)) ## Embedding layer
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [13]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=early_stopping)

Epoch 1/10
625/625 [==============================] - 168s 266ms/step - loss: 13423255552.0000 - accuracy: 0.5820 - val_loss: 0.6070 - val_accuracy: 0.6686
Epoch 2/10
625/625 [==============================] - 197s 314ms/step - loss: 0.5188 - accuracy: 0.7595 - val_loss: 0.5359 - val_accuracy: 0.7212
Epoch 3/10
625/625 [==============================] - 214s 342ms/step - loss: 0.6211 - accuracy: 0.8158 - val_loss: 0.5079 - val_accuracy: 0.7312
Epoch 4/10
625/625 [==============================] - 117s 186ms/step - loss: 0.2769 - accuracy: 0.8837 - val_loss: 0.4106 - val_accuracy: 0.8158
Epoch 5/10
625/625 [==============================] - 81s 129ms/step - loss: 0.1775 - accuracy: 0.9313 - val_loss: 0.4364 - val_accuracy: 0.8222
Epoch 6/10
625/625 [==============================] - 87s 140ms/step - loss: 0.1236 - accuracy: 0.9524 - val_loss: 0.4742 - val_accuracy: 0.8222
Epoch 7/10
625/625 [==============================] - 85s 137ms/step - loss: 0.0911 - accuracy: 0.9671 - val_loss: 0

In [14]:
model.save('simple_rnn_imdb.h5')